# Основы веб-скрапинга и работы с API

In [52]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs

## Задание 1

In [82]:
KEYWORDS = ['Linux', 'Go']

In [83]:
# Надали пройтись по всем страницам? ничего не сказано?

def parse_page(url):
    res = requests.get(url)

    soup = bs(res.text, 'html.parser')
    
    articles = pd.DataFrame()

    for post in soup.find_all('article', class_='post_preview'):
        link = post.find('a', class_='post__title_link')
        time = post.find('span', class_='post__time').text
        article_text = post.find('div', class_='post__text').text

        row = {
            'time': time,
            'title': link.text,
            'link': link['href'],
            'text': article_text
        }

        articles = pd.concat([articles, pd.DataFrame([row])])
        
    return articles

articles = parse_page('https://habr.com/ru/all/')

articles.head()

articles[articles['text'].str.contains('|'.join(KEYWORDS))].head(10)

,time,title,link,text
0,сегодня в 18:27,Основы Bash-скриптинга для непрограммистов,https://habr.com/ru/post/538942/,"\nСтатья рассчитана на тех, кто не имеет или и..."
0,сегодня в 17:56,Резервное копирование данных в домашних условиях,https://habr.com/ru/post/538938/,"\n Статья описывает, как в домашних услов..."
0,сегодня в 17:52,Релиз ruleguard v0.3.0,https://habr.com/ru/post/538930/,"\nА что, если я скажу вам, что линтеры для Go ..."
0,сегодня в 17:40,Зачем нужен обратный прокси сервер в 5 актах,https://habr.com/ru/post/538936/,\nНа текущий момент есть большое разнообразие ...
0,сегодня в 16:57,"«Если хочешь разобраться, найдешь возможность»...",https://habr.com/ru/company/spbifmo/blog/538918/,"\nМихаил Мирзаянов, разработавший платформу Co..."


## Задание 2

In [85]:
emails = ['fake@gmail.com', 'fake2@gmail.com']

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

payload = {"emailAddresses": emails}

headers = {
    'Content-Type': 'application/json;charset=utf-8',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

res = requests.post(url, json=payload, headers=headers)

In [86]:
json_res = json.loads(res.text)

breaches = pd.DataFrame()

for email, summary in json_res['summary'].items():
    for breach_id in summary['breaches']:
        breach = json_res['breaches'][str(breach_id)]

        row = {
            'email': email,
            'datetime': pd.to_datetime(breach['publishDate']).date(),
            'source': breach['site'],
            'description': breach['description']
        }

        breaches = pd.concat([breaches, pd.DataFrame([row])])

breaches.head(30)

,email,datetime,source,description
0,fake@gmail.com,2019-09-26,cprewritten.net,"In July 2019, virtual world Club Penguin Rewri..."
0,fake@gmail.com,2019-04-04,avvo.com,"In May 2018, the legal marketplace Avvo's data..."
0,fake@gmail.com,2019-03-20,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,fake@gmail.com,2019-03-20,roll20.net,"In January 2019, Roll20 allegedly breached. Th..."
0,fake@gmail.com,2020-10-08,unknown,"In February 2017, the Tanzanian sports betting..."
0,fake@gmail.com,2020-12-10,eskuvoszombathely.hu,"In November 2020, a collection of over 23,000 ..."
0,fake@gmail.com,2019-07-04,modaoperandi.com,"In March 2019, online fashion retailer Moda Op..."
0,fake@gmail.com,2020-05-14,minted.com,"In May 2020, the hacking group “ShinyHunters” ..."
0,fake@gmail.com,2020-12-10,lifebeast.com,"In November 2020, a collection of over 23,000 ..."
0,fake@gmail.com,2020-11-19,glofox.com,"At an unconfirmed date, the gym management sof..."
